In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_7/models/B5R2b5_5FP_1FC_fold7_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image4/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold4.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold4.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold7.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(657, 25)
(657, 25)
Normal:  (429, 25)
Abnormal:  (228, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,411,411,49,P5,P52,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,200.0,119.0,561.0,667.0,0.202166,0.132937,0.564982,0.743028,AB01 P5-2 C049.JPG,AB01 P5-2 C049.JPG
1,1,1,412,412,49,P2,P2,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,191.0,125.0,453.0,472.0,0.189902,0.139520,0.450450,0.525253,AB02 P2 C049.JPG,AB02 P2 C049.JPG
2,2,2,413,413,49,P5,P51,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,162.0,163.0,546.0,638.0,0.157665,0.181944,0.530864,0.709091,AB02 P5-1 C049.JPG,AB02 P5-1 C049.JPG
3,3,3,414,414,49,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,271.0,118.0,530.0,603.0,0.275638,0.131439,0.538745,0.670707,AB02 P1 C049.JPG,AB02 P1 C049.JPG
4,4,4,415,415,49,P3,P31,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,149.0,152.0,577.0,718.0,0.164463,0.169823,0.634730,0.797980,AB02 P3-1 C049.JPG,AB02 P3-1 C049.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 657 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.17397553, -0.1774632 ,  0.43369696, ...,  0.77547967,
         0.78780884, -0.11987032],
       [ 0.04323596,  0.57785118, -0.18428168, ..., -0.1163874 ,
        -0.16957082,  0.77788275],
       [-0.15690885, -0.19504572,  0.34766412, ...,  0.30007991,
         0.10092686, -0.11217467],
       ...,
       [-0.18144682, -0.17663722,  0.56200784, ...,  0.36148557,
         0.53769052, -0.14221232],
       [-0.1411927 , -0.2132621 ,  0.6862964 , ...,  0.34421834,
         0.26959273, -0.16657363],
       [ 0.00855663, -0.00626816, -0.01926091, ..., -0.12524293,
        -0.16805226, -0.05761572]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.173976,-0.177463,0.433697,0.100809,-0.128020,-0.120812,-0.150787,0.219978,0.335342,-0.118003,...,0.208618,0.075357,0.628716,0.105769,0.326435,-0.175498,-0.195850,0.775480,0.787809,-0.119870
1,0.043236,0.577851,-0.184282,0.418325,0.579166,-0.040894,0.556999,0.358772,0.070719,0.027367,...,0.051514,-0.135189,-0.207488,-0.107270,-0.173138,0.823604,0.550526,-0.116387,-0.169571,0.777883
2,-0.156909,-0.195046,0.347664,0.556901,-0.097679,-0.162777,-0.148226,0.047419,0.114956,-0.200555,...,0.360969,0.148434,0.068999,0.125204,0.220005,-0.161138,-0.219117,0.300080,0.100927,-0.112175
3,-0.038033,0.699824,-0.076951,0.501103,0.602246,-0.021318,0.225625,0.605132,-0.073835,0.084560,...,0.056104,-0.196376,-0.177179,0.009061,-0.145393,0.583961,0.250047,-0.084115,-0.215513,0.816456
4,0.305695,0.313379,-0.145852,-0.077773,0.058768,0.874340,-0.097994,0.634504,-0.012574,0.846872,...,0.335608,-0.069345,0.227140,0.662677,-0.141696,0.398475,0.293077,0.137701,0.051385,0.880231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,-0.026421,-0.165454,-0.208760,-0.044831,-0.018537,-0.125202,-0.210612,-0.099431,-0.141840,0.242555,...,0.095156,-0.176710,0.032534,0.353936,-0.164277,0.301787,-0.224241,-0.031287,-0.095794,0.041673
653,-0.086747,-0.155141,-0.068611,0.098978,-0.053980,0.450030,-0.148814,0.391194,0.122927,0.219781,...,0.490093,0.126354,0.183453,0.066935,-0.177498,-0.180776,0.089382,0.317262,0.239469,0.463252
654,-0.181447,-0.176637,0.562008,0.283786,-0.067406,-0.210617,-0.211688,-0.146790,0.360520,-0.218080,...,0.103112,0.307700,0.195643,0.001283,0.179503,-0.164411,-0.202408,0.361486,0.537691,-0.142212
655,-0.141193,-0.213262,0.686296,0.379133,0.137676,-0.156439,-0.174663,-0.066849,0.717559,-0.160175,...,-0.009886,0.481852,0.553011,0.398252,0.873334,-0.220589,-0.227014,0.344218,0.269593,-0.166574


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.173976,-0.177463,0.433697,0.100809,-0.128020,-0.120812,-0.150787,0.219978,0.335342,-0.118003,...,0.326435,-0.175498,-0.195850,0.775480,0.787809,-0.119870,Abnormal,P52,FP-C,AB01
1,0.043236,0.577851,-0.184282,0.418325,0.579166,-0.040894,0.556999,0.358772,0.070719,0.027367,...,-0.173138,0.823604,0.550526,-0.116387,-0.169571,0.777883,Abnormal,P2,FP-A,AB02
2,-0.156909,-0.195046,0.347664,0.556901,-0.097679,-0.162777,-0.148226,0.047419,0.114956,-0.200555,...,0.220005,-0.161138,-0.219117,0.300080,0.100927,-0.112175,Abnormal,P51,FP-C,AB02
3,-0.038033,0.699824,-0.076951,0.501103,0.602246,-0.021318,0.225625,0.605132,-0.073835,0.084560,...,-0.145393,0.583961,0.250047,-0.084115,-0.215513,0.816456,Abnormal,P1,FP-A,AB02
4,0.305695,0.313379,-0.145852,-0.077773,0.058768,0.874340,-0.097994,0.634504,-0.012574,0.846872,...,-0.141696,0.398475,0.293077,0.137701,0.051385,0.880231,Abnormal,P31,FP-B,AB02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,-0.026421,-0.165454,-0.208760,-0.044831,-0.018537,-0.125202,-0.210612,-0.099431,-0.141840,0.242555,...,-0.164277,0.301787,-0.224241,-0.031287,-0.095794,0.041673,Normal,P32,FP-B,Normal
653,-0.086747,-0.155141,-0.068611,0.098978,-0.053980,0.450030,-0.148814,0.391194,0.122927,0.219781,...,-0.177498,-0.180776,0.089382,0.317262,0.239469,0.463252,Normal,P42,FP-B,Normal
654,-0.181447,-0.176637,0.562008,0.283786,-0.067406,-0.210617,-0.211688,-0.146790,0.360520,-0.218080,...,0.179503,-0.164411,-0.202408,0.361486,0.537691,-0.142212,Normal,P52,FP-C,Normal
655,-0.141193,-0.213262,0.686296,0.379133,0.137676,-0.156439,-0.174663,-0.066849,0.717559,-0.160175,...,0.873334,-0.220589,-0.227014,0.344218,0.269593,-0.166574,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_5FP_RF_5FP_fold7_1.csv')